## 검색 결과 융합(RAG Fusion)과 응답 품질 향상

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

In [5]:
# 🔸 LangChain v1.0 기준 Fusion RAG + Answer Refine Agent 예제
# pip install langchain langchain-openai langchain-community faiss-cpu python-dotenv

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool
from langchain.agents import create_agent

load_dotenv()

# -------------------------------------------------------
# 1) LLM 및 임베딩 모델 초기화
# -------------------------------------------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# -------------------------------------------------------
# 2) 샘플 문서 및 벡터스토어 초기화
# -------------------------------------------------------
docs = [
    Document(page_content="인공지능은 인간의 학습 능력과 추론을 모방하는 기술이다."),
    Document(page_content="머신러닝은 데이터를 이용해 스스로 패턴을 학습하는 인공지능의 한 분야이다."),
    Document(page_content="RAG는 검색과 생성 모델을 결합해 응답의 정확성을 높인다."),
    Document(page_content="Fusion RAG는 여러 검색 결과를 결합해 응답 품질을 높이는 방법이다."),
]
vectorstore = FAISS.from_documents(docs, embeddings)

# -------------------------------------------------------
# 3) Fusion RAG 도구 정의
# -------------------------------------------------------
@tool
def fusion_rag_search(query: str) -> str:
    """Fusion RAG 방식으로 여러 쿼리 검색 결과를 융합하여 문맥을 반환한다."""
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    sub_queries = [
        query,
        f"{query} 관련 개념을 설명하라",
        f"{query}의 작동 원리를 요약하라",
    ]

    all_results = []
    for q in sub_queries:
        results = retriever.invoke(q)
        all_results.extend(results)

    # 중복 제거
    unique_texts = list({d.page_content for d in all_results})
    fused_context = "\n".join(unique_texts)

    return f"[융합 문맥]\n{fused_context}"

# -------------------------------------------------------
# 4) 응답 품질 향상 도구 정의
# -------------------------------------------------------
@tool
def answer_refiner(context: str, question: str) -> str:
    """Fusion된 문맥을 기반으로 고품질 응답을 생성한다."""
    prompt = f"""
    아래 문맥을 참고하여 질문에 대해 명확하고 근거 있는 답변을 작성하라.

    [문맥]
    {context}

    [질문]
    {question}

    답변:
    """
    response = llm.invoke(prompt)
    return response.content

# -------------------------------------------------------
# 5) 에이전트 생성 (LangChain v1.0 형식)
# -------------------------------------------------------
agent = create_agent(
    model=llm,
    tools=[fusion_rag_search, answer_refiner],
    system_prompt="사용자의 요청을 해결하기 위해 필요시 Fusion RAG과 품질 향상 도구를 사용하라."
)

# -------------------------------------------------------
# 6) 실행: v1에서는 messages 리스트로 전달
# -------------------------------------------------------
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "RAG Fusion이 무엇이며 응답 품질을 어떻게 향상시키는가?"}
    ]
})

print("\n=== 최종 응답 ===")
print(result["messages"][-1].content)


=== 최종 응답 ===
RAG Fusion은 정보 검색의 한 방법으로, 여러 검색 결과를 융합하여 문맥을 생성하는 기술입니다. 이 기술은 대규모 데이터셋에서 관련 정보를 효과적으로 추출하고, 이를 바탕으로 더 정확하고 유의미한 응답을 생성하는 데 중점을 둡니다. RAG는 "Retrieval-Augmented Generation"의 약자로, 검색(retrieval)과 생성(generation) 모델을 결합하여 작동합니다.

RAG Fusion이 응답 품질을 향상시키는 방법은 다음과 같습니다:

1. **정보의 다양성**: 여러 검색 결과를 통합하여 다양한 관점을 포함한 응답을 생성합니다. 이는 사용자가 요청한 정보에 대해 보다 포괄적이고 균형 잡힌 답변을 제공할 수 있게 합니다.

2. **정확성 향상**: 검색된 정보의 정확성을 높임으로써, 생성된 응답의 신뢰성을 강화합니다. RAG는 최신 정보와 관련된 데이터를 검색하여, 사용자가 필요로 하는 정확한 정보를 기반으로 응답을 생성합니다.

3. **문맥 이해**: 검색된 정보를 바탕으로 문맥을 형성하여, 단순한 사실 나열이 아닌, 사용자의 질문에 대한 깊이 있는 이해를 바탕으로 한 응답을 제공합니다. 이는 사용자가 원하는 정보를 보다 명확하게 전달하는 데 기여합니다.

4. **적응성**: 다양한 데이터 소스에서 정보를 검색하고 융합할 수 있기 때문에, 변화하는 정보 환경에 적응하여 최신 정보를 반영한 응답을 생성할 수 있습니다.

결론적으로, RAG Fusion은 검색과 생성 모델의 결합을 통해 정보의 정확성과 다양성을 높이고, 문맥을 이해하여 보다 유의미한 응답을 생성함으로써 응답 품질을 향상시키는 기술입니다.


### RAG Fusion활용 예제

In [6]:
"""
🔸 LangChain v1.0 기준 Fusion RAG + Answer Refine Agent 예제
검색 결과 융합(RAG Fusion)과 응답 품질 향상 에이전트 구현

주요 기능:
1. Fusion RAG: 여러 쿼리로 검색하여 결과를 융합
2. Answer Refine Agent: 융합된 문맥을 기반으로 고품질 응답 생성
3. LangChain v1.0 에이전트 프레임워크 활용
"""

import os
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage

# 환경 변수 로드
load_dotenv()

class FusionRAGAgent:
    """Fusion RAG와 Answer Refine Agent를 결합한 클래스"""
    
    def __init__(self, model_name: str = "gpt-4o-mini", embedding_model: str = "text-embedding-3-small"):
        """
        Fusion RAG Agent 초기화
        
        Args:
            model_name: 사용할 LLM 모델명
            embedding_model: 사용할 임베딩 모델명
        """
        self.llm = ChatOpenAI(model=model_name, temperature=0.2)
        self.embeddings = OpenAIEmbeddings(model=embedding_model)
        self.vectorstore = None
        self.agent = None
        
    def setup_knowledge_base(self, documents: List[Document]):
        """
        지식 베이스 설정
        
        Args:
            documents: 벡터스토어에 저장할 문서 리스트
        """
        print("📚 지식 베이스 구축 중...")
        self.vectorstore = FAISS.from_documents(documents, self.embeddings)
        print(f"✅ {len(documents)}개 문서가 벡터스토어에 저장되었습니다.")
        
    def create_fusion_rag_tool(self):
        """Fusion RAG 검색 도구 생성"""
        
        @tool
        def fusion_rag_search(query: str) -> str:
            """
            Fusion RAG 방식으로 여러 쿼리 검색 결과를 융합하여 문맥을 반환한다.
            
            Args:
                query: 검색할 질문
                
            Returns:
                융합된 문맥 정보
            """
            if not self.vectorstore:
                return "벡터스토어가 초기화되지 않았습니다."
                
            retriever = self.vectorstore.as_retriever(search_kwargs={"k": 3})
            
            # 다양한 관점의 서브 쿼리 생성
            sub_queries = [
                query,
                f"{query} 관련 개념을 설명하라",
                f"{query}의 작동 원리를 요약하라",
                f"{query}의 장단점은 무엇인가",
                f"{query}의 실제 활용 사례는 무엇인가"
            ]
            
            all_results = []
            for q in sub_queries:
                try:
                    results = retriever.invoke(q)
                    all_results.extend(results)
                except Exception as e:
                    print(f"쿼리 '{q}' 검색 중 오류: {e}")
                    continue
            
            # 중복 제거 및 문맥 융합
            unique_texts = list({d.page_content for d in all_results})
            fused_context = "\n".join(unique_texts)
            
            return f"[융합 문맥]\n{fused_context}\n\n[검색된 문서 수: {len(unique_texts)}개]"
        
        return fusion_rag_search
    
    def create_answer_refiner_tool(self):
        """응답 품질 향상 도구 생성"""
        
        @tool
        def answer_refiner(context: str, question: str) -> str:
            """
            Fusion된 문맥을 기반으로 고품질 응답을 생성한다.
            
            Args:
                context: 융합된 문맥 정보
                question: 원본 질문
                
            Returns:
                개선된 응답
            """
            prompt = f"""
            아래 문맥을 참고하여 질문에 대해 명확하고 근거 있는 답변을 작성하라.
            
            답변 작성 시 다음 사항을 고려하라:
            1. 문맥의 정보를 충분히 활용하라
            2. 구체적이고 실용적인 예시를 포함하라
            3. 명확한 구조로 답변을 구성하라
            4. 불확실한 정보는 명시하라
            
            [문맥]
            {context}
            
            [질문]
            {question}
            
            답변:
            """
            
            try:
                response = self.llm.invoke(prompt)
                return response.content
            except Exception as e:
                return f"응답 생성 중 오류가 발생했습니다: {e}"
        
        return answer_refiner
    
    def create_agent(self):
        """Fusion RAG 에이전트 생성"""
        
        # 도구 생성
        fusion_rag_tool = self.create_fusion_rag_tool()
        answer_refiner_tool = self.create_answer_refiner_tool()
        
        # 시스템 프롬프트 정의
        system_prompt = """
        당신은 Fusion RAG와 Answer Refine Agent를 활용하는 지능형 질의응답 시스템입니다.
        
        작업 순서:
        1. 사용자의 질문을 분석한다
        2. fusion_rag_search 도구를 사용해 관련 문맥을 검색하고 융합한다
        3. answer_refiner 도구를 사용해 융합된 문맥을 기반으로 고품질 응답을 생성한다
        4. 최종 응답을 사용자에게 제공한다
        
        항상 근거 있는 답변을 제공하고, 불확실한 정보는 명시하라.
        """
        
        # 에이전트 생성
        self.agent = create_agent(
            model=self.llm,
            tools=[fusion_rag_tool, answer_refiner_tool],
            system_prompt=system_prompt
        )
        
        print("🤖 Fusion RAG 에이전트가 생성되었습니다.")
    
    def query(self, question: str) -> str:
        """
        질문에 대한 응답 생성
        
        Args:
            question: 사용자 질문
            
        Returns:
            에이전트의 응답
        """
        if not self.agent:
            return "에이전트가 초기화되지 않았습니다. create_agent()를 먼저 호출하세요."
        
        try:
            result = self.agent.invoke({
                "messages": [
                    HumanMessage(content=question)
                ]
            })
            
            # 마지막 AI 메시지의 내용 반환
            for message in reversed(result["messages"]):
                if isinstance(message, AIMessage):
                    return message.content
            
            return "응답을 생성할 수 없습니다."
            
        except Exception as e:
            return f"질의응답 처리 중 오류가 발생했습니다: {e}"


def create_sample_documents() -> List[Document]:
    """샘플 문서 생성"""
    return [
        Document(page_content="인공지능(AI)은 인간의 학습 능력과 추론을 모방하는 기술이다. 머신러닝, 딥러닝, 자연어처리 등의 하위 분야를 포함한다."),
        Document(page_content="머신러닝은 데이터를 이용해 스스로 패턴을 학습하는 인공지능의 한 분야이다. 지도학습, 비지도학습, 강화학습으로 분류된다."),
        Document(page_content="RAG(Retrieval-Augmented Generation)는 검색과 생성 모델을 결합해 응답의 정확성과 신뢰성을 높이는 기술이다."),
        Document(page_content="Fusion RAG는 여러 검색 결과를 결합해 응답 품질을 높이는 방법이다. 다양한 관점의 쿼리를 통해 더 포괄적인 정보를 수집한다."),
        Document(page_content="벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 시스템이다. FAISS, Pinecone, Weaviate 등이 대표적이다."),
        Document(page_content="임베딩은 텍스트를 고차원 벡터로 변환하는 기술이다. 의미적 유사성을 벡터 공간에서 측정할 수 있게 해준다."),
        Document(page_content="LangChain은 LLM 애플리케이션 개발을 위한 프레임워크이다. 체인, 에이전트, 메모리 등의 기능을 제공한다."),
        Document(page_content="에이전트는 도구를 사용해 복잡한 작업을 수행하는 AI 시스템이다. 계획 수립, 도구 선택, 실행, 평가의 사이클을 반복한다."),
    ]


def main():
    """메인 실행 함수"""
    print("🚀 Fusion RAG + Answer Refine Agent 예제 시작")
    print("=" * 50)
    
    # 1. Fusion RAG Agent 초기화
    agent = FusionRAGAgent()
    
    # 2. 샘플 문서로 지식 베이스 구축
    documents = create_sample_documents()
    agent.setup_knowledge_base(documents)
    
    # 3. 에이전트 생성
    agent.create_agent()
    
    # 4. 테스트 질문들
    test_questions = [
        "RAG Fusion이 무엇이며 응답 품질을 어떻게 향상시키는가?",
        "머신러닝과 딥러닝의 차이점은 무엇인가?",
        "벡터 데이터베이스의 역할과 장점은 무엇인가?",
        "LangChain 에이전트의 작동 원리를 설명하라"
    ]
    
    print("\n📝 테스트 질문 실행")
    print("=" * 50)
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n🔍 질문 {i}: {question}")
        print("-" * 30)
        
        response = agent.query(question)
        print(f"💡 응답: {response}")
        print("-" * 50)
    
    # 5. 대화형 모드
    print("\n💬 대화형 모드 (종료하려면 'quit' 입력)")
    print("=" * 50)
    
    while True:
        try:
            user_input = input("\n질문을 입력하세요: ").strip()
            
            if user_input.lower() in ['quit', 'exit', '종료']:
                print("👋 프로그램을 종료합니다.")
                break
            
            if not user_input:
                continue
                
            print("\n🤔 답변 생성 중...")
            response = agent.query(user_input)
            print(f"\n💡 응답: {response}")
            
        except KeyboardInterrupt:
            print("\n👋 프로그램을 종료합니다.")
            break
        except Exception as e:
            print(f"\n❌ 오류가 발생했습니다: {e}")


if __name__ == "__main__":
    main()

# 질문 예시
# 랭체인이 뭐지?
# RAG의 기능은?
# 임베딩은?

🚀 Fusion RAG + Answer Refine Agent 예제 시작
📚 지식 베이스 구축 중...
✅ 8개 문서가 벡터스토어에 저장되었습니다.
🤖 Fusion RAG 에이전트가 생성되었습니다.

📝 테스트 질문 실행

🔍 질문 1: RAG Fusion이 무엇이며 응답 품질을 어떻게 향상시키는가?
------------------------------
💡 응답: RAG Fusion은 Retrieval-Augmented Generation의 한 형태로, 여러 검색 결과를 통합하여 응답의 품질을 높이는 방법입니다. 이 기술은 다양한 관점에서 쿼리를 수행하여 더 포괄적이고 정확한 정보를 수집하는 데 중점을 둡니다.

### RAG Fusion의 작동 원리

1. **다양한 쿼리 수행**: RAG Fusion은 사용자가 입력한 질문에 대해 여러 가지 관점에서 쿼리를 생성합니다. 예를 들어, "기후 변화의 원인은 무엇인가?"라는 질문에 대해 "기후 변화의 과학적 원인", "사회적 요인", "경제적 영향" 등 다양한 측면에서 정보를 검색합니다.

2. **검색 결과 통합**: 각 쿼리에서 얻은 검색 결과를 결합하여, 중복된 정보는 제거하고 서로 보완적인 정보를 통합합니다. 이를 통해 보다 풍부하고 다각적인 응답을 생성할 수 있습니다.

3. **응답 생성**: 통합된 정보를 바탕으로 최종 응답을 생성합니다. 이 과정에서 RAG는 생성 모델을 활용하여 자연스러운 언어로 응답을 구성하며, 정보의 정확성과 신뢰성을 높입니다.

### 응답 품질 향상 방법

- **정보의 포괄성**: 다양한 쿼리를 통해 수집된 정보는 단일 출처에서 얻은 정보보다 더 포괄적입니다. 예를 들어, 기후 변화에 대한 질문에 대해 여러 관점에서 정보를 제공함으로써 사용자는 보다 깊이 있는 이해를 할 수 있습니다.

- **신뢰성 증가**: 여러 출처에서 수집된 정보를 통합함으로써, 특정 출처의 오류나 편향을 줄일 수 있습니다. 이는 사용자가 보다 신뢰할 수 있는 정보를 제공받는 데 기여합니다.

-


질문을 입력하세요:  랭체인이 뭐지?



🤔 답변 생성 중...

💡 응답: 랭체인(LangChain)은 대형 언어 모델(LLM) 애플리케이션 개발을 위한 강력한 프레임워크입니다. 이 프레임워크는 개발자들이 다양한 AI 작업을 보다 효율적으로 수행할 수 있도록 돕는 여러 기능을 제공합니다. 주요 기능은 다음과 같습니다:

1. **체인(Chain)**: 여러 작업을 순차적으로 연결하여 실행할 수 있는 기능입니다. 예를 들어, 사용자가 질문을 입력하면, 랭체인은 질문을 이해하고 정보를 검색한 후 최종적으로 답변을 생성하는 과정을 체인으로 구성할 수 있습니다.

2. **에이전트(Agent)**: 특정 작업을 수행하기 위해 필요한 여러 단계를 자동으로 처리하는 기능입니다. 고객 지원 챗봇을 개발할 때, 에이전트는 사용자의 질문을 분석하고 적절한 답변을 찾거나 필요한 경우 다른 시스템에 연결하여 정보를 가져오는 등의 작업을 수행할 수 있습니다.

3. **메모리(Memory)**: 이전의 대화나 작업 내용을 기억하여 사용자와의 상호작용을 개인화하고 일관되게 만들어 줍니다. 예를 들어, 사용자가 이전에 문의했던 내용을 기억하고 후속 질문을 할 때 더 정확하고 관련성 높은 답변을 제공할 수 있습니다.

이러한 기능들을 통해 랭체인은 개발자들이 복잡한 AI 애플리케이션을 보다 쉽게 구축하고 관리할 수 있도록 지원합니다. 자연어 처리(NLP)와 관련된 다양한 프로젝트에서 유용하게 사용될 수 있으며, 자동화된 고객 서비스, 콘텐츠 생성, 데이터 분석 등의 분야에서 활용될 수 있습니다. 

결론적으로, 랭체인은 LLM을 활용한 애플리케이션 개발을 위한 유용한 도구로, 다양한 기능을 통해 개발자들이 보다 효율적으로 작업할 수 있도록 돕습니다.



질문을 입력하세요:  RAG의 기능은?



🤔 답변 생성 중...

💡 응답: RAG(Retrieval-Augmented Generation)의 기능은 주로 두 가지로 요약할 수 있습니다: 정보 검색과 생성 모델의 결합을 통한 응답의 정확성 및 신뢰성 향상입니다. 이를 통해 RAG는 사용자가 요청하는 정보에 대해 보다 포괄적이고 신뢰할 수 있는 답변을 제공합니다.

1. **정보 검색 기능**: RAG는 사용자의 쿼리에 대해 관련된 정보를 검색하는 기능을 가지고 있습니다. 예를 들어, 사용자가 "기후 변화의 원인"에 대한 질문을 했을 때, RAG는 다양한 데이터베이스나 문서에서 기후 변화와 관련된 정보를 검색하여 그 결과를 기반으로 응답을 생성합니다. 이 과정에서 여러 관점의 정보를 수집하여 보다 균형 잡힌 답변을 제공합니다.

2. **응답 생성 기능**: 검색된 정보를 바탕으로 RAG는 자연어 생성 모델을 활용하여 사용자가 이해하기 쉬운 형태로 응답을 작성합니다. 예를 들어, 검색된 정보가 "온실가스 배출", "산업화", "산림 파괴"와 같은 여러 원인에 대한 내용이라면, RAG는 이를 종합하여 "기후 변화는 주로 온실가스 배출, 산업화, 그리고 산림 파괴와 같은 여러 요인에 의해 발생합니다."와 같은 형태로 응답을 생성할 수 있습니다.

결론적으로, RAG는 정보 검색과 생성의 두 가지 기능을 결합하여 사용자가 원하는 정보를 보다 정확하고 신뢰성 있게 제공하는 기술입니다. 이러한 기능은 특히 복잡한 질문에 대한 답변을 요구하는 상황에서 유용하게 활용될 수 있습니다.



질문을 입력하세요:  임베딩은?



🤔 답변 생성 중...

💡 응답: 임베딩(Embedding)은 텍스트, 이미지, 오디오 등 다양한 형태의 데이터를 고차원 벡터로 변환하는 기술입니다. 이 과정에서 각 데이터는 특정한 의미를 가지는 벡터로 표현되며, 이러한 벡터는 데이터 간의 의미적 유사성을 벡터 공간에서 측정할 수 있게 해줍니다. 즉, 비슷한 의미를 가진 데이터는 벡터 공간에서 가까운 위치에 배치되고, 서로 다른 의미를 가진 데이터는 멀리 떨어지게 됩니다.

### 구체적인 예시
예를 들어, "고양이"와 "강아지"라는 단어의 임베딩 벡터는 서로 가까운 위치에 있을 것입니다. 반면 "고양이"와 "자동차"의 벡터는 멀리 떨어져 있을 것입니다. 이러한 임베딩은 자연어 처리(NLP) 분야에서 단어의 의미를 이해하고, 문장 간의 유사성을 평가하는 데 매우 유용합니다.

### 임베딩의 활용
임베딩은 검색 엔진, 추천 시스템, 감정 분석 등 다양한 분야에서 활용됩니다. 예를 들어, 추천 시스템에서는 사용자의 과거 행동 데이터를 임베딩하여 비슷한 취향을 가진 다른 사용자와의 유사성을 기반으로 콘텐츠를 추천할 수 있습니다.

### 벡터 데이터베이스
임베딩된 벡터를 효율적으로 저장하고 검색하기 위해 벡터 데이터베이스가 사용됩니다. FAISS, Pinecone, Weaviate와 같은 시스템은 대량의 고차원 벡터를 빠르게 검색할 수 있는 기능을 제공합니다. 이러한 데이터베이스는 대규모 데이터셋에서 유사한 항목을 찾는 데 필수적입니다.

결론적으로, 임베딩은 데이터의 의미를 수치적으로 표현하고, 이를 통해 다양한 응용 프로그램에서 데이터 간의 관계를 이해하고 활용할 수 있게 해주는 중요한 기술입니다.



질문을 입력하세요:  quit


👋 프로그램을 종료합니다.
